# Практическая часть курсовой работы

## 1. Предсказание вычислительной сложности блока

В первой части мы будем взаимодействовать с сетью Ethereum (sepolia) через библиотеку web3.py

Для работы с блокчейном требуется точка доступа к узлу сети Ethereum. 
Существует несколько способов подключения к узлам, самый простой - использовать провайдер https://www.web3gate.ru/.  

Получите ключ и сохраните в файле `.env` в корне проекта. 
Пример того, как должны храниться ключи, приведен в файле `.env.example`  

Для доступа к приватным переменным используется библиотека [python-dotenv](https://pypi.org/project/python-dotenv/)  

In [36]:
from dotenv import dotenv_values

config = dotenv_values(".env")
WEB3_API_KEY = config["WEB3_API_KEY"]

web3url = f'https://b.web3gate.ru/{WEB3_API_KEY}'

Библиотека `web3.py` позволяет взаимодействовать с блокчейном Ethereum через указанный провайдер. 

In [38]:
from web3 import Web3 

provider = Web3.HTTPProvider(web3url)
w3 = Web3(provider)
w3.is_connected()

True

Чтение баланса указанного адреса возвращает результат в единицах wei. 
Для перевода в единицы ether используется метод `from_wei`. 

In [9]:
wallet_address = '0xeE2d4c1EF974a67E16caD4B19F209697694B4010'

wei_balance = w3.eth.get_balance(wallet_address)
eth_balance = Web3.from_wei(wei_balance, 'ether')
print(f"Balance of {wallet_address}={eth_balance} Eth")

balance of 0xeE2d4c1EF974a67E16caD4B19F209697694B4010=0.900263432268376196 Eth


Простейшие транзакции - перемещение криптовалюты от адреса 1 к адресу 2. 
Кроме того, в Ethereum есть более сложные транзакции, за выполнение которых требуется уплатить комиссию. 
Газ — это единица для измерения объема вычислительных усилий, необходимых для выполнения определенных операций в сети Ethereum. Подробнее - https://ethereum.org/ru/developers/docs/gas/   

По мере поступления новых транзакций от участников сети, формируются новые блоки.  

Проверим содержимое последнего блока с помощью кода. 

In [58]:
w3.eth.get_block('latest') 

AttributeDict({'baseFeePerGas': 1131584548,
 'blobGasUsed': 655360,
 'difficulty': 0,
 'excessBlobGas': 0,
 'extraData': HexBytes('0x'),
 'gasLimit': 35964845,
 'gasUsed': 88382,
 'hash': HexBytes('0xa00d0b9fbf617cafa48069da7fbed3e4a30dda548da9f443cb9fccdd32d15464'),
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000220000000000000000200000000010000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000008000000000000000004000000000000000000000000000000000000000002000000004000000000000000000000020000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000001000000000000000000000010000000000080000000000000000000000000000000000000000000000000'),
 'miner': '0x0F35B0753E261375C9a6Cb44316b4BdC7e765509',
 'mixHash': HexBytes('0x6276708f8bf33d09373b294433135acf5842c5028e129ae2988d09da79debe93'),
 'nonce': HexBytes('0x0000000000000000'),


Мы видим список транзакций в поле 'transactions'. 
Посмотреть содержимое транзакции можно по ее хешу. 

In [ ]:
txn_hash = '0x1c328747541b5e3c86df34888e3d0d3f5e36c81566d14929e076487929d4e5aa'
txn = w3.eth.get_transaction(txn_hash)

In [57]:
txn

AttributeDict({'blockHash': HexBytes('0x442761891e176afe4016de3817deb9cb1656b9bb926b25ee7c29e5b72ccce517'),
 'blockNumber': 5654577,
 'from': '0xeE2d4c1EF974a67E16caD4B19F209697694B4010',
 'gas': 21000,
 'gasPrice': 3504689041,
 'maxFeePerGas': 4447945144,
 'maxPriorityFeePerGas': 1500000000,
 'hash': HexBytes('0x1c328747541b5e3c86df34888e3d0d3f5e36c81566d14929e076487929d4e5aa'),
 'input': HexBytes('0x'),
 'nonce': 3,
 'to': '0xf39Fd6e51aad88F6F4ce6aB8827279cffFb92266',
 'transactionIndex': 37,
 'value': 1000000000000000,
 'type': 2,
 'accessList': [],
 'chainId': 11155111,
 'v': 1,
 'r': HexBytes('0xd3ca4c281d8fe397ebd191dc77fe498a2525315194c4d667d73b1744b169bfa9'),
 's': HexBytes('0x2c53e143c9153272e07299d6e29d69b29cdd9513979c6fc897eb2b29587189cb'),
 'yParity': 1})

Давайте предскажем, как зависит количество газа в блоке от времени суток.  

### Задание 1
Прочитайте содержимое 100 блоков (например, 5654477 - 5654577)  и постройте модель линейной регрессии временного ряда.  

**X** - `timestamp` - время валидации блока.  

**Y** - `gasUsed` - количество использованного газа в блоке.  

Постройте график предсказанных и фактических значений. Ось X - астрономическое время. 

## 2. Предварительная обработка транзакций 
Библиотека `web3py` хорошо подходит для несложных запросов и совершения транзакций. Однако для поиска нужной информации в блокчейне потребуется совершить слишком много запросов.  

Для чтения содержимого блокчейна есть более подходящие инструменты, например обозреватель https://sepolia.etherscan.io/. 
У него удобный пользовательский интерфейс, исследуйте его.  

Кроме того, платформа предоставляет API для выполнения http-запросов. 
Бесплатный ключ можно получить после регистрации, на странице https://etherscan.io/myapikey  
Получите ключ и вставьте его в файл `.env`. 

In [41]:
config = dotenv_values(".env")
ETHERSCAN_API_KEY = config["ETHERSCAN_API_KEY"]

Получим историю транзакций по конкретному адресу

In [45]:
query = f'https://api-sepolia.etherscan.io/api?\
module=account&\
action=txlist&\
address={wallet_address}&\
startblock=0&\
endblock=99999999&\
page=1&\
offset=10&\
sort=asc&\
apikey={ETHERSCAN_API_KEY}'

In [46]:
import requests
data = requests.get(query)

In [43]:
data.json()

{'status': '1',
 'message': 'OK',
 'result': [{'blockNumber': '4380515',
   'timeStamp': '1695889704',
   'hash': '0xa944042f17ea4b45139323887ec31f278acbf253268aa51166609e7aa98935e0',
   'nonce': '85823',
   'blockHash': '0xe735192212e1f9eaf7a5dd8622f0b16c6751372372130d8568c323b50d2b026e',
   'transactionIndex': '0',
   'from': '0x2cda41645f2dbffb852a605e92b185501801fc28',
   'to': '0xee2d4c1ef974a67e16cad4b19f209697694b4010',
   'value': '50000000000000000',
   'gas': '21000',
   'gasPrice': '100000000000',
   'isError': '0',
   'txreceipt_status': '1',
   'input': '0x',
   'contractAddress': '',
   'cumulativeGasUsed': '21000',
   'gasUsed': '21000',
   'confirmations': '3259680',
   'methodId': '0x',
   'functionName': ''},
  {'blockNumber': '4380537',
   'timeStamp': '1695889992',
   'hash': '0x286a923450f7683f0185afacf0d39909be379abd96a7a148adbd1df62dbf90c5',
   'nonce': '0',
   'blockHash': '0x1c8831e314036cbf26181faa8b902854b68e1b1fd8deb40c16dd1c2f3ed4d4bb',
   'transactionIndex

In [49]:
data.json()

{'status': '1',
 'message': 'OK',
 'result': [{'blockNumber': '4380515',
   'timeStamp': '1695889704',
   'hash': '0xa944042f17ea4b45139323887ec31f278acbf253268aa51166609e7aa98935e0',
   'nonce': '85823',
   'blockHash': '0xe735192212e1f9eaf7a5dd8622f0b16c6751372372130d8568c323b50d2b026e',
   'transactionIndex': '0',
   'from': '0x2cda41645f2dbffb852a605e92b185501801fc28',
   'to': '0xee2d4c1ef974a67e16cad4b19f209697694b4010',
   'value': '50000000000000000',
   'gas': '21000',
   'gasPrice': '100000000000',
   'isError': '0',
   'txreceipt_status': '1',
   'input': '0x',
   'contractAddress': '',
   'cumulativeGasUsed': '21000',
   'gasUsed': '21000',
   'confirmations': '3259970',
   'methodId': '0x',
   'functionName': ''},
  {'blockNumber': '4380537',
   'timeStamp': '1695889992',
   'hash': '0x286a923450f7683f0185afacf0d39909be379abd96a7a148adbd1df62dbf90c5',
   'nonce': '0',
   'blockHash': '0x1c8831e314036cbf26181faa8b902854b68e1b1fd8deb40c16dd1c2f3ed4d4bb',
   'transactionIndex

### Задание 2 
Для анализа транзакций с помощью методов машинного обучения необходима предварительная обработка данных.  
Преобразуйте полученный ответ во взвешенный граф в форме списка транзакций.   
Тип данных транзакции - [namedtuple](https://docs.python.org/3/library/collections.html#collections.namedtuple).  

In [56]:
from collections import namedtuple

Trx = namedtuple('Transaction', ['from_', 'to_', 'value'])

# for example
t = Trx('0x2cda41645f2dbffb852a605e92b185501801fc28', '0xac6074d4a18137f3aa7d1ed47dd06e7fac543130', '1000000000000000')
t.from_

'0x2cda41645f2dbffb852a605e92b185501801fc28'